In [ ]:
import tweepy
import os

In [2]:
auth = tweepy.AppAuthHandler(os.environ.get('TWITTER_API_KEY'),
                             os.environ.get('TWITTER_API_SECRET'))
# Construct the API instance
api = tweepy.API(auth)

# Basic Queries

## By Content

In [24]:
vt = api.search(q='url:virustotal.com', lang='en')
ha = api.search(q='url: hybrid-analysis.com', lang='en')
vb = api.search(q='url:virusbay.io', lang='en')

print(f"got {len(vt)} tweets with VirusTotal, {len(ha)} with Hybrid-Analysis, and {len(vb)} with VirusBay")

got 14 tweets with VirusTotal, 0 with Hybrid-Analysis, and 14 with VirusBay


## Querying users' timelines

In [11]:
accounts = ["@botNET__","@anyrun_app","@JohnLaTwC","@B_H101","@buffaloverflow",
            "@dvk01uk","@JakubKroustek","@malwrhunterteam","@MalwareHunterBR",
            "@SevenLayerJedi","@James_inthe_box","@malware_traffic","@MaliciousDelish",
            "@GossiTheDog","@hexlax","@0Btemos_BHS","@SecurityAura","@_odisseus"]


items = []
for account in accounts:
    try:
        items.extend(tweepy.Cursor(api.user_timeline, id=account).items(100)) # Max 100 tweets per user
    except Exception as e:
        print(account, e)

len(items)

@MalwareHunterBR Twitter error response: status code = 404


1608

 ## Single Tweet Format

In [25]:
sample_tweet = vt[1]

sample_tweet.__dict__

{'_api': <tweepy.api.API at 0x10342ef28>,
 '_json': {'contributors': None,
  'coordinates': None,
  'created_at': 'Tue Apr 09 09:59:18 +0000 2019',
  'entities': {'hashtags': [{'indices': [15, 24], 'text': 'Trickbot'}],
   'symbols': [],
   'urls': [{'display_url': 'virustotal.com/en/file/2a8ee2…',
     'expanded_url': 'https://www.virustotal.com/en/file/2a8ee2a8578ef1453761c9ccb6b54387373dbf55e6478a50b38d436607a126aa/analysis/1554768356/',
     'indices': [126, 149],
     'url': 'https://t.co/f1bkPSDMhY'}],
   'user_mentions': [{'id': 3069922981,
     'id_str': '3069922981',
     'indices': [3, 13],
     'name': 'avman',
     'screen_name': 'avman1995'}]},
  'favorite_count': 0,
  'favorited': False,
  'geo': None,
  'id': 1115554773493846016,
  'id_str': '1115554773493846016',
  'in_reply_to_screen_name': None,
  'in_reply_to_status_id': None,
  'in_reply_to_status_id_str': None,
  'in_reply_to_user_id': None,
  'in_reply_to_user_id_str': None,
  'is_quote_status': False,
  'lang': '

### We only need the _json part

In [26]:
sample_tweet._json

{'contributors': None,
 'coordinates': None,
 'created_at': 'Tue Apr 09 09:59:18 +0000 2019',
 'entities': {'hashtags': [{'indices': [15, 24], 'text': 'Trickbot'}],
  'symbols': [],
  'urls': [{'display_url': 'virustotal.com/en/file/2a8ee2…',
    'expanded_url': 'https://www.virustotal.com/en/file/2a8ee2a8578ef1453761c9ccb6b54387373dbf55e6478a50b38d436607a126aa/analysis/1554768356/',
    'indices': [126, 149],
    'url': 'https://t.co/f1bkPSDMhY'}],
  'user_mentions': [{'id': 3069922981,
    'id_str': '3069922981',
    'indices': [3, 13],
    'name': 'avman',
    'screen_name': 'avman1995'}]},
 'favorite_count': 0,
 'favorited': False,
 'geo': None,
 'id': 1115554773493846016,
 'id_str': '1115554773493846016',
 'in_reply_to_screen_name': None,
 'in_reply_to_status_id': None,
 'in_reply_to_status_id_str': None,
 'in_reply_to_user_id': None,
 'in_reply_to_user_id_str': None,
 'is_quote_status': False,
 'lang': 'en',
 'metadata': {'iso_language_code': 'en', 'result_type': 'recent'},
 'pla

## Example for timed fetching(blocking)

In [35]:
from time import sleep
from datetime import datetime
results = []
since_id = None
delay = 60 * 5 # 5 minutes

In [ ]:
while True:
    vt = api.search(q='url:virustotal.com', lang='en', since_id=since_id)
    print(f"{datetime.now().strftime('%m/%d/%Y %H:%M:%S')}: got {len(vt)} new tweets")
    results.extend(vt)
    since_id = vt.max_id # set the max_id for the next query to only get tweets with ids that are higher(newer tweets)
    sleep(delay) # Sleep for 5 minutes

## Saving the tweets for later analysis

In [43]:
with open('fetched_tweets.json', 'w') as f:
    all_tweets = list(map(lambda x: x._json, [*vt, *vb, *ha, *items]))
    json.dump(all_tweets, f)